In [20]:
import numpy as np
import pandas as pd
import pylogit as pl
import os, sys
import warnings
warnings.filterwarnings("ignore")
from geopy.distance import geodesic
from tqdm import tqdm
# switch folder
root_dir = "/home/juanwu/Projects/micro-mobility/beam/test/input/sfbay"
os.chdir(root_dir)
os.getcwd()

'/home/juanwu/Projects/micro-mobility/beam/test/input/sfbay'

# Reconstruction of Logit Model with urbansim data

In [2]:
data_dir = os.path.join(root_dir, "gemini/activitysim-plans-base-2010-cut-60k/")
# households
hh = pd.read_csv(os.path.join(data_dir, "households.csv.gz"))
display('Households: ',hh.columns.to_numpy(), hh.head(5))
# parcels (land-use types, useless)
#par = pd.read_csv(os.path.join(data_dir, "parcels.csv.gz"))
#display('Parcel: ', par.columns.to_numpy(), par.head(5))
# persons
per = pd.read_csv(os.path.join(data_dir, "persons.csv.gz"))
display('Persons: ',per.columns.to_numpy(), per.head(5))
# plans
plns = pd.read_csv(os.path.join(data_dir, "plans.csv.gz")).sort_values(by='person_id')
display('Plans: ', plns.columns.to_numpy(), plns.head(5))
# units (useless for modeling)
# unts = pd.read_csv(os.path.join(data_dir, "units.csv.gz"))
# display('Units: ', unts.columns, unts.head(5))

'Households: '

array(['Unnamed: 0', 'household_id', 'serialno', 'persons', 'cars',
       'income', 'race_of_head', 'age_of_head', 'workers', 'children',
       'tenure', 'recent_mover', 'block_id'], dtype=object)

,Unnamed: 0,household_id,serialno,persons,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,block_id
0,21,2017,2.012001e+12,1,1,95000,1,36,1,0,2,1,60014002002004
1,63,6051,2.009000e+12,1,1,60000,1,34,1,0,2,1,60014004003000
2,126,12099,2.010000e+12,1,1,38500,1,50,1,0,2,0,60014009001010
3,163,15651,2.013001e+12,1,1,421000,1,50,1,0,2,1,60014011001013
4,402,38597,2.011000e+12,1,1,9800,2,62,0,0,2,0,60014034004001


'Persons: '

array(['Unnamed: 0', 'person_id', 'age', 'earning', 'edu', 'hours',
       'household_id', 'member_id', 'race_id', 'relate', 'sex', 'student',
       'work_at_home', 'worker'], dtype=object)

,Unnamed: 0,person_id,age,earning,edu,hours,household_id,member_id,race_id,relate,sex,student,work_at_home,worker
0,42,4116,36,95000,21,55,2017,1,1,0,2,0,0,1
1,134,12089,34,36000,22,30,6051,1,1,0,2,0,0,1
2,279,24344,50,38500,16,40,12099,1,1,0,1,0,1,1
3,345,30891,50,421000,22,50,15651,1,1,0,1,0,0,1
4,755,73298,62,0,18,0,38597,1,2,0,2,0,0,0


'Plans: '

array(['Unnamed: 0', 'Unnamed: 0.1', 'trip_id', 'person_id',
       'number_of_participants', 'trip_mode', 'PlanElementIndex',
       'ActivityElement', 'ActivityType', 'x', 'y', 'departure_time'],
      dtype=object)

,Unnamed: 0,Unnamed: 0.1,trip_id,person_id,number_of_participants,trip_mode,PlanElementIndex,ActivityElement,ActivityType,x,y,departure_time
0,3279,1824,NaN,394.0,NaN,NaN,1,activity,Home,-122.234022,37.865274,18.979
1,3280,1825,129433.0,394.0,1.0,SHARED2FREE,2,leg,NaN,NaN,NaN,NaN
2,3281,1825,NaN,394.0,NaN,NaN,3,activity,othdiscr,-122.677004,38.363021,20.633
3,3282,1826,129437.0,394.0,1.0,SHARED2PAY,4,leg,NaN,NaN,NaN,NaN
4,3283,1826,NaN,394.0,NaN,NaN,5,activity,Home,-122.234022,37.865274,NaN


## Preprocessing Plans data

In [3]:
# Calculate distance
dist = []
ref_id = 0
for i in tqdm(range(len(plns)-1)):
    if plns.person_id[i]==ref_id and plns.person_id[i+1]==ref_id:
        if plns.ActivityElement[i]=='leg':
            x_1, y_1 = plns.x[i-1], plns.y[i-1]
            x_2, y_2 = plns.x[i+1], plns.y[i+1]
            dist.append(geodesic((y_1, x_1), (y_2,x_2)).mi)
        else:
            dist.append(np.nan)
    else:
        ref_id = plns.person_id[i]
        dist.append(np.nan)
dist.append(np.nan)
plns['distance']=dist
display(plns.distance.unique())

100%|██████████| 532195/532195 [01:13<00:00, 7281.41it/s]


array([        nan, 41.96719154, 10.63982756, ...,  3.34258917,
       90.89298593, 22.53065104])

In [7]:
# encode mode
plns_proc = plns.dropna(subset=['trip_mode'])
mode_map = {
        "BIKE": 0,
        "DRIVEALONEFREE": 1,
        "DRIVEALONEPAY": 1,
        "DRIVE_COM": 2,
        "DRIVE_EXP": 2,
        "DRIVE_HVY": 2,
        "DRIVE_LOC": 2,
        "DRIVE_LRF": 2,
        "SHARED2FREE": 1,
        "SHARED2PAY": 1,
        "SHARED3FREE": 1,
        "SHARED3PAY": 1,
        "TAXI": 3,
        "TNC_SHARED": 3,
        "TNC_SINGLE": 3,
        "WALK": 4,
        "WALK_COM": 5,
        "WALK_EXP": 5,
        "WALK_HVY": 5,
        "WALK_LOC": 5,
        "WALK_LRF": 5,
}
plns_proc.trip_mode = plns_proc.trip_mode.apply(lambda x: mode_map[x])
print("After mapping, only %d of data persisted" % len(plns_proc))
plns_proc.head(5)

After mapping, only 236023 of data persisted


,Unnamed: 0,Unnamed: 0.1,trip_id,person_id,number_of_participants,trip_mode,PlanElementIndex,ActivityElement,ActivityType,x,y,departure_time,distance
1,3280,1825,129433.0,394.0,1.0,1,2,leg,NaN,NaN,NaN,NaN,41.967192
3,3282,1826,129437.0,394.0,1.0,1,4,leg,NaN,NaN,NaN,NaN,41.967192
6,3285,1828,129873.0,395.0,1.0,1,2,leg,NaN,NaN,NaN,NaN,10.639828
8,3287,1829,129877.0,395.0,1.0,1,4,leg,NaN,NaN,NaN,NaN,10.639828
13,3292,1832,130157.0,396.0,1.0,1,4,leg,NaN,NaN,NaN,NaN,NaN


## Join datasets

In [22]:
ref_list = plns.person_id.astype(int)
joinable_id = []
# extract joinable ids
for id_ in per.person_id:
    if id_ in ref_list:
        joinable_id.append(id_)
print("Joinable" if len(joinable_id)>0 else "Unjoinable")
print(len(joinable_id))
del joinable_id

Joinable
4981


In [35]:
pln_attrs = ['person_id', 'trip_mode', 'distance']
drop_cols = ['Unnamed: 0_x', 'Unnamed: 0_y', 'serialno', 
                 'person_id', 'household_id', 'recent_mover', 'block_id',
                    'member_id', 'relate', 'workers', 'children',
                        'race_of_head', 'age_of_head']
per_hh = pd.merge(hh, per, on='household_id')
data = pd.merge(plns_proc[pln_attrs], per_hh, on='person_id')
data = data.drop(drop_cols, axis=1)
print("Extracted %d rows of data" % len(data))
display(data.head(5))

Extracted 236023 rows of data


,trip_mode,distance,persons,cars,income,tenure,age,earning,edu,hours,race_id,sex,student,work_at_home,worker
0,1,41.967192,3,2,285000,1,0,0,0,0,1,1,0,0,0
1,1,41.967192,3,2,285000,1,0,0,0,0,1,1,0,0,0
2,1,10.639828,3,2,285000,1,40,210000,23,60,1,1,0,0,1
3,1,10.639828,3,2,285000,1,40,210000,23,60,1,1,0,0,1
4,1,NaN,3,2,285000,1,38,75000,23,40,1,2,0,0,0


## Logit Model

In [42]:
import torch as T
import torch.nn as nn
import torch.optim as optim

def one_hot_encode(int_mode: np.ndarray, total: int):
    res = []
    for k in int_mode:
        empty_choice = np.zeros(total)
        empty_choice[k] += 1
        res.append(empty_choice)
    return res
# use neural network to approximate utility functions
## length of inputs and outputs
in_dim, out_dim = len(data.columns) - 1, 6
## extract 
X = data.loc[:, data.columns!='trip_mode'].to_numpy()
y = data.trip_mode.to_numpy()
y = one_hot_encode(y, 6)

In [47]:
# neural network
lyrs = nn.Sequential(
    nn.Linear(in_dim, out_dim),
    nn.Softmax()
)
# training
X = T.tensor(X, dtype=T.float32)
y = T.tensor(y, dtype=T.float32)
y_pred = lyrs(X[:4]) 
loss = nn.GaussianNLLLoss()
var = torch.ones(5, 2, requires_grad=True)
loss(y_pred, y[:4], var)

AttributeError: module 'torch.nn' has no attribute 'GaussianNLLLoss'

In [48]:
T.__version__

'1.7.0'